In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from itertools import combinations
from scipy import stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import math

In [51]:
#import test and train datasets

In [52]:
df_train = pd.read_csv('/Users/lakshikajain/Documents/GitHub/DataScienceCapstone_2/output.csv')
df_test = pd.read_csv('/Users/lakshikajain/Documents/GitHub/DataScienceCapstone_2/Testing.csv')

In [53]:
#dropped Fluid_overload as all rows for this field is 0

In [54]:
df_train.drop('fluid_overload', axis = 1, inplace = True)

In [55]:
df_train_2 = df_train.loc[:, df_train.columns != "prognosis"]

In [56]:
df_train_2 = df_train_2.apply(pd.to_numeric, errors = 'coerce')

In [19]:
df_train = pd.concat([df_train_2,df_train['prognosis']], axis=1, join='inner')

In [ ]:
# Fit the model

In [24]:
X = df_train.drop('prognosis', axis = 1)
y = df_train['prognosis'].values
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 2.50132708,  2.29336913,  6.6749948 , -0.2173802 , -0.14981285,
        -0.43999471, -0.40183714, -0.2173802 , -0.2173802 , -0.14981285,
        -0.14981285, -0.79795147, -0.21428571, -0.14981285, -0.80410593,
        -0.15401412, -0.15401412, -0.15401412, -0.22043899, -0.31960998,
        -0.22043899, -0.31960998, -0.14981285, -0.15401412, -0.35982855,
        -0.61870779, -0.14981285, -0.31728716, -0.39978778, -0.14981285,
        -0.2173802 , -0.54728837, -0.47701668, -0.36198685, -0.55105045,
        -0.55293081, -0.15811388, -0.22043899, -0.22043899, -0.51520103,
        -0.35982855, -0.27844132, -0.15401412, -0.44590404, -0.15401412,
        -0.15401412, -0.27589035, -0.40795783, -0.27332236, -0.27844132,
        -0.15811388, -0.15811388, -0.15811388, -0.15811388, -0.15811388,
        -0.40592207, -0.14981285, -0.2234636 , -0.15401412, -0.15401412,
        -0.15401412, -0.15401412, -0.22043899, -0.27073683, -0.15401412,
        -0.15401412, -0.22043899, -0.15401412, -0.1

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01, random_state = 0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4870, 131) (4870,)
(50, 131) (50,)


In [ ]:
#K Nearest Neighbor

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
Ks=20
mean_acc=np.zeros((Ks-1))
std_acc=np.zeros((Ks-1))
for n in range(1,Ks):
    neigh = KNeighborsClassifier(n_neighbors=n).fit(X_train, y_train)
    y_hat_KNN = neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, y_hat_KNN)
    std_acc[n-1] = np.std(y_hat_KNN == y_test)/np.sqrt(y_hat_KNN.shape[0])

print('best accuracy :', mean_acc.max(), ' |  Best K is:', mean_acc.argmax()+1)

best accuracy : 1.0  |  Best K is: 1


In [27]:
neigh_forTest = KNeighborsClassifier(n_neighbors=mean_acc.argmax()+1).fit(X, y)
neigh_forTest

KNeighborsClassifier(n_neighbors=1)

In [28]:
X_train_DT, X_test_DT, y_train_DT, y_test_DT = train_test_split(X,y,test_size=0.3,random_state=1)

print("X_train size :", X_train_DT.shape, "  y_train size :", y_train_DT.shape)
print("X_test size  :", X_test_DT.shape, "  y_test size  :", y_test_DT.shape)

X_train size : (3444, 131)   y_train size : (3444,)
X_test size  : (1476, 131)   y_test size  : (1476,)


In [ ]:
#Decision Tree Classifer

In [29]:
from sklearn.tree import DecisionTreeClassifier
Ks=200
mean_acc=np.zeros((Ks-1))
for n in range(1,Ks):
    DecTree = DecisionTreeClassifier(criterion="entropy", max_depth=n).fit(X_train_DT, y_train_DT)
    pred_Tree = DecTree.predict(X_test_DT)
    mean_acc[n-1] = metrics.accuracy_score(y_test_DT, pred_Tree)

print('best accuracy :', mean_acc.max(), ' |  Best K is:', mean_acc.argmax()+1)

best accuracy : 1.0  |  Best K is: 24


In [30]:
DecTree_forTest = DecisionTreeClassifier(criterion="entropy", max_depth=mean_acc.argmax()+1).fit(X, y)
DecTree_forTest

DecisionTreeClassifier(criterion='entropy', max_depth=24)

In [35]:
X_train_LR, X_test_LR, y_train_LR, y_test_LR = train_test_split(X,y,test_size=0.3,random_state=3)

print("X_train size :", X_train_LR.shape, "  y_train size :", y_train_LR.shape)
print("X_test size  :", X_test_LR.shape, "  y_test size  :", y_test_LR.shape)

C=np.arange(1,0.001, step=-0.001).tolist()

X_train size : (3444, 131)   y_train size : (3444,)
X_test size  : (1476, 131)   y_test size  : (1476,)


In [ ]:
#Logistic Regression 

In [36]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression().fit(X_train_LR, y_train_LR)
yhat = LR.predict(X_test_LR)
mean_acc = metrics.accuracy_score(y_test_LR, yhat)

print('best accuracy :', mean_acc, ' |  Best c is:', )

best accuracy : 1.0  |  Best c is:


In [37]:
LR_forTest = LogisticRegression().fit(X, y)
LR_forTest

LogisticRegression()

In [ ]:
# Apply models on Test data

In [39]:
test_df = df_test
test_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [ ]:
#fluid overload field has the same 0 for all rows, thus remove this field

In [40]:
test_df.loc[:, (test_df == 0).all()]

,fluid_overload
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [41]:
test_df2 = test_df.loc[:, test_df.columns != "prognosis"]
test_df2 = test_df2.apply(pd.to_numeric, errors = 'coerce')
test_df = pd.concat([test_df2,test_df['prognosis']], axis=1, join='inner')
test_df = test_df.dropna()

In [42]:
test_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [43]:
test_df_no_fluid = test_df.drop('fluid_overload', axis = 1)

In [44]:
test_df_no_fluid = test_df.drop('fluid_overload', axis = 1)
test_df_no_fluid.shape

(42, 132)

In [45]:
Feature_test_no_fluid = test_df_no_fluid.drop('prognosis',axis=1)
Feature_test_no_fluid.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
X_testing_no_fluid = preprocessing.StandardScaler().fit(Feature_test_no_fluid).transform(Feature_test_no_fluid)
X_testing_no_fluid = np.nan_to_num(X_testing_no_fluid)
X_testing_no_fluid.shape

(42, 131)

In [47]:

y_testing = test_df['prognosis'].values
y_testing.shape

(42,)

In [48]:
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [58]:
#KNN
y_hat_KNN_test_no_fluid = neigh_forTest.predict(X_testing_no_fluid)
print("KNN_f1_score :",f1_score(y_testing, y_hat_KNN_test_no_fluid, average='weighted'))
print("\n")

#Decision Tree
y_hat_DecTree_test_no_fluid = DecTree_forTest.predict(X_testing_no_fluid)
print("DecTree_f1_score :",f1_score(y_testing, y_hat_DecTree_test_no_fluid, average='weighted'))
print("\n")

#Logistic Regression
y_hat_LR_test_no_fluid = LR_forTest.predict(X_testing_no_fluid)
y_hat_LR_test_prob_no_fluid = LR_forTest.predict_proba(X_testing_no_fluid)
print("LR_f1_score :",f1_score(y_testing, y_hat_LR_test_no_fluid, average='weighted'))
print("LR_LogLoss  :", log_loss(y_testing, y_hat_LR_test_prob_no_fluid))

KNN_f1_score : 0.9761904761904762


DecTree_f1_score : 1.0


LR_f1_score : 0.9761904761904762
LR_LogLoss  : 0.0573592687430264


In [ ]:
#Decision Tree performed better than the KNN and Logistic Regression models